In [ ]:
!pip install nltk

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install textblob

In [ ]:
import pandas as pd
import re
import string
import emoji
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Preprocesar el texto eliminando URLs, menciones, emojis y caracteres especiales
def clean_text(text):
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Eliminar menciones de usuarios (@)
    text = re.sub(r'\@\w+', '', text)
    # Eliminar emojis
    text = emoji.replace_emoji(text, replace="")
    # Eliminar signos de puntuación y números
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convertir a minúsculas
    text = text.lower()
    return text

In [ ]:
df = pd.read_csv("training_1600000_processed_noemoticon.csv", encoding='latin-1')
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
df

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df['cleaned_text'] = df['text'].apply(clean_text)

In [ ]:
import nltk
import math
from textblob import TextBlob
nltk.download('stopwords')

In [ ]:
df = df[['target', 'text','date', 'cleaned_text']]
# C) Longitud del tweet (número de caracteres)
df['tweet_length'] = df['cleaned_text'].apply(len)
# D) Frecuencia de emojis
def count_emojis(text):
    return sum(1 for char in text if char in emoji.EMOJI_DATA)

df['emoji_count'] = df['text'].apply(count_emojis)
# E) Uso de signos de exclamación/interrogación
df['exclamation_count'] = df['text'].apply(lambda x: x.count('!'))
df['question_count'] = df['text'].apply(lambda x: x.count('?'))

# Función para calcular la proporción de palabras en mayúsculas en un tweet
def capital_word_density(text):
    words = text.split()
    if len(words) == 0:
        return 0
    uppercase_words = [word for word in words if word.isupper()]
    return len(uppercase_words) / len(words)

df['capital_word_density'] = df['text'].apply(capital_word_density)
# Calcular la cantidad de palabras en cada tweet
df['word_count'] = df['cleaned_text'].apply(lambda x: len(x.split()))

# Contar el número de stopwords en cada tweet
stop_words = set(stopwords.words('english'))

def stopword_count(text):
    words = text.split()
    stopword_count = sum(1 for word in words if word in stop_words)
    return stopword_count

df['stopword_count'] = df['cleaned_text'].apply(stopword_count)
# Función para contar el número de palabras únicas en cada tweet
df['unique_word_count'] = df['cleaned_text'].apply(lambda x: len(set(x.split())))
# Calcular la proporción de palabras repetidas
def repeated_word_proportion(text):
    words = text.split()
    if len(words) == 0:
        return 0
    unique_words = set(words)
    return (len(words) - len(unique_words)) / len(words)

df['repeated_word_proportion'] = df['cleaned_text'].apply(repeated_word_proportion)

# Sentimiento global usando TextBlob
df['textblob_polarity'] = df['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['textblob_subjectivity'] = df['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
# Contar la cantidad de signos de puntuación
df['punctuation_count'] = df['text'].apply(lambda x: len([char for char in x if char in string.punctuation]))
# Contar el número de menciones y hashtags
df['mention_count'] = df['text'].apply(lambda x: x.count('@'))
df['hashtag_count'] = df['text'].apply(lambda x: x.count('#'))

def calculate_entropy(text):
    prob = [text.count(c) / len(text) for c in set(text)]
    entropy = -sum([p * math.log(p) for p in prob])
    return entropy

df['text_entropy'] = df['cleaned_text'].apply(calculate_entropy)
# Función para detectar sarcasmo basado en el uso de exclamaciones y puntuaciones
def detect_sarcasm(text):
    if '!' in text and analyzer.polarity_scores(text)['neg'] > 0.5:
        return 1
    return 0

df['sarcasm'] = df['text'].apply(detect_sarcasm)

<ipython-input-8-2f9311adc476>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_length'] = df['cleaned_text'].apply(len)
<ipython-input-8-2f9311adc476>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emoji_count'] = df['text'].apply(count_emojis)


In [ ]:
df.to_csv('datatotal.csv')